# Visualized Performanced of the Modified Informer

In [8]:
import torch
import torch.nn as nn
from tpinform_experiment import TPTrans, PositionalEncoding
from dataloader import load_parquet, preprocess_data, SlidingWindowDataset
from plotting import plot_testresult_sample
from torch.utils.data import DataLoader

## Short Distances

In [14]:
df = load_parquet("dataset", k=5)
df = preprocess_data(df)

dataset = SlidingWindowDataset(
    df,
    max_diff_per_sequence_minutes=15,
    window_size_minutes=120,
    pred_size_minutes=30,
    stride=15,
)
print("Number of windows:", len(dataset))
test_loader = DataLoader(dataset, batch_size=1, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.load(
    "checkpoints/t_1500_400_20_420_120_15_64_0.0005.pth",
    map_location=device,
    weights_only=False
)

model.eval()

criterion = nn.MSELoss()
test_loss = 0.0
data = []

with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)

        # don’t pass tgt_start, your current forward() doesn’t use it
        pred = model(x)

        data.append((x, y, pred))

        # if you trained on coordinates, be consistent:
        # loss = criterion(deltas_to_coords(x, pred), deltas_to_coords(x, y))
        loss = criterion(pred, y)

        test_loss += loss.item() * x.size(0)

test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.6f}", flush=True)

plot_testresult_sample(data)

/zhome/63/7/219953/DeepLearning-1/dataloader.py:167: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df.set_index("Timestamp")
/zhome/63/7/219953/DeepLearning-1/dataloader.py:167: FutureWarning: DataFrameGroupBy.resample operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.set_index("Timestamp")


Number of windows: 12


AttributeError: 'TPTrans' object has no attribute 'decoder'